In [33]:
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from implementations import *

In [2]:
train_set = load_csv_data('/Users/bedanian/Desktop/Machine Learning/Project 1/train.csv', sub_sample = True)
test_set = load_csv_data('/Users/bedanian/Desktop/Machine Learning/Project 1/test.csv', sub_sample = False)

In [29]:
#train set test set and id
x_train = train_set[1]
y_train = train_set[0]
ids_train = train_set[2]

x_test = test_set[1]
y_test = test_set[0]
ids_test = test_set[2]

In [47]:
#function to preprocess training and test set 
def standardize(x_train, x_test):
    mean = np.mean(x_train)
    norm = np.linalg.norm(x_train)
    x_train_std = (x_train - mean)/norm
    x_test_std = (x_test - mean)/norm
    return x_train_std, x_test_std

def sigmoid(x):
    s = 0.5 * (1 + np.tanh(0.5*x))
    return s

def zero_to_neg(array):
    ret = np.zeros(len(array))
    for i, v in enumerate(array):
        if v == 0:
            ret[i] = -1
        else:
            ret[i] = v
    return ret

def build_poly(x, degree):
    poly = x
    for deg in range(2, degree+1):
        poly = np.concatenate((poly, np.power(x, deg)), axis = 1)
    return poly

In [48]:
#drop columns with 70% of NA 
na_indices = get_na_columns(x_train, 0.7, -999)
x_train = np.delete(x_train, na_indices, axis = 1)
x_test = np.delete(x_test, na_indices, axis = 1)
#standardize 
x_train_0_std, x_test_0_std = standardize(x_train, x_test)
#apply polynome of degree 3 on it
x_train_0_std = build_poly(x_train_0_std, degree = 3)
x_test_0_std = build_poly(x_test_0_std, degree = 3)

In [50]:
#ridge regression with lbd 10**-11 need to be explain why 
(w,loss) = ridge_regression(y_train, x_train_0_std, 10**(-11))
#use sigmoid to put the value between 1 and 0 
# if 0 put -1
y = zero_to_neg(np.around(sigmoid(x_test_0_std @ w)))

In [51]:
#submit
create_csv_submission(ids_test, y, 'prediction.csv')